In [6]:
# data processing
# x_train is like past n days, y_train is future n days
# past n days will be 365 ? future n days is 7 ? for the week?
import pickle
import numpy as np
import pandas as pd
infile = open('cleaned_aqi_data.pkl', 'rb')
aqi_data = pickle.load(infile)
training_set = pd.DataFrame(aqi_data['AQI'])

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )


In [ ]:
# data visualization

In [ ]:
# data standardizing

In [8]:
# model: recurrent neural network

# x_train and y_train should be numpy arrays

# https://medium.com/analytics-vidhya/weather-forecasting-with-recurrent-neural-networks-1eaa057d70c3

from keras.models import Sequential
from keras.layers import Bidirectional, LSTM,Dense ,Dropout


In [9]:

# set up regressor model
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc']) # adam optimizer for optimal efficiency
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )


Epoch 1/500
65/65 [==============================] - 28s 166ms/step - loss: 0.0279 - acc: 0.2425
Epoch 2/500
65/65 [==============================] - 8s 129ms/step - loss: 0.0168 - acc: 0.2604
Epoch 3/500
 9/65 [===>..........................] - ETA: 5s - loss: 0.0153 - acc: 0.2292

KeyboardInterrupt: ignored

In [ ]:
# read test dataset
testdataset = y_test
real_temperature = y_test
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

# predict model
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))
